# source

> Functions that emit markdown from the `source` element of various notebook cell types

In [ ]:
#| default_exp source

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
from fastcore.test import *

In [ ]:
#| hide
#| export
import io
from typing import Iterable, Optional, Sequence, Tuple

In [ ]:
# | exporti


def is_directive_line(line: str):
    return line.startswith('#|') or line.startswith('# |')


def parse_directive_line(line: str) -> Tuple[str, Optional[bool]]:
    assert is_directive_line(line)

    directive = line.lstrip('# |').strip()
    parts = [part.strip() for part in directive.split(':')]

    # A directive is either a single key or key: value
    assert len(parts) == 1 or len(parts) == 2
    key, *value_list = parts
    value_str: Optional[str] = value_list[0] if value_list else None

    # Deal with string forms of true and false
    # These directives are technically YAML, so allowing all the values
    # for bool per [the spec](https://yaml.org/type/bool.html).

    true_vals = [
        'y',
        'Y',
        'yes',
        'Yes',
        'YES',
        'true',
        'True',
        'TRUE',
        'on',
        'On',
        'ON',
    ]
    false_vals = [
        'n',
        'N',
        'no',
        'No',
        'NO',
        'false',
        'False',
        'FALSE',
        'off',
        'Off',
        'OFF',
    ]

    value: Optional[bool] = None
    if value_str in true_vals:
        value = True
    elif value_str in false_vals:
        value = False

    return key, value

In [ ]:
#| hide 
test_cases = [
    {
        'line': '#| hide',
        'expected': ('hide', None)
    },
    {
        'line': '#| echo: true',
        'expected': ('echo', True)
    },
    {
        'line': '# | echo: false',
        'expected': ('echo', False)
    },
    {
        'line': '# | code-fold: On',
        'expected': ('code-fold', True)
    },
]

for tc in test_cases:
    actual = parse_directive_line(tc['line'])
    test_eq(actual, tc['expected'])

In [ ]:
# | export
def emit_python_source(source: Sequence[str], stream: io.TextIOBase):
    # Extract directives
    directives = {}
    i = 0 # initialize explicitly because `source` may be empty 
    for i, line in enumerate(source):
        if is_directive_line(line):
            key, value = parse_directive_line(line)
            directives[key] = value
        else:
            break

    # Handle directives per https://quarto.org/docs/reference/cells/cells-jupyter.html#code-output
    # and https://quarto.org/docs/reference/cells/cells-jupyter.html#cell-output.

    should_echo = 'echo' not in directives or directives['echo']
    should_show_output = 'output' not in directives or directives['output']

    if should_echo:
        stream.write('```python\n')
        for line in source[i:]:
            stream.write(line)
        stream.write('\n```\n\n')

    return should_show_output

In [ ]:
python_source = ['name = \'world\'\n', 'print(f"hello, {name}")']
stream = io.StringIO()

should_show_output = emit_python_source(python_source, stream)

stream.seek(0)
output = stream.read()

expected = """\
```python
name = 'world'
print(f"hello, {name}")
```

"""

test_eq(output, expected)
test_eq(should_show_output, True)
print(output)

```python
name = 'world'
print(f"hello, {name}")
```




In [ ]:
# Test use of directives
python_source = ['# | echo: false', '# | output: false', 'name = \'world\'\n', 'print(f"hello, {name}")']
stream = io.StringIO()

should_show_output = emit_python_source(python_source, stream)

stream.seek(0)
output = stream.read()

expected = ''

test_eq(output, expected)
test_eq(should_show_output, False)

In [ ]:
#| export
def emit_markdown_source(markdown: Iterable[str], stream: io.TextIOBase):
    for line in markdown:
        stream.write(line)
    stream.write('\n')

In [ ]:
markdown_source = [
    '# This is Markdown\n',
    'This is a [link](https://www.google.com).\n',
    'This is:\n',
    '\n',
    '* a \n',
    '* bulleted\n',
    '* list\n',
    '\n',
    'Yup.',
]
stream = io.StringIO()

emit_markdown_source(markdown_source, stream)

stream.seek(0)
output = stream.read()

expected = """\
# This is Markdown
This is a [link](https://www.google.com).
This is:

* a 
* bulleted
* list

Yup.
"""

test_eq(output, expected)
print(output)

# This is Markdown
This is a [link](https://www.google.com).
This is:

* a 
* bulleted
* list

Yup.



In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()